## Example pipeline (one OSCAR file, no streaming)

Things to change when working in different environment

In [1]:
# dependencies
from datasets import load_dataset, DatasetDict
import os
from transformers import AutoTokenizer, RobertaTokenizerFast
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
import torch
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

# data files
path = '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_short.txt'

In [2]:
# Check that PyTorch sees it
print(f'Cuda available? {torch.cuda.is_available()}')

Cuda available? False


In [3]:
# def create_text_generator(gen):
#     for i in gen:
#         yield i['text']



# oscar_short = load_dataset('text', data_files={"train": path}, split='train')
# oscar_short_it = load_dataset('text', data_files={"train": path}, split='train', streaming=True)

# p = '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/nl_part_1.txt'

# oscar1_it = load_dataset('text', data_files={"train": p}, split='train', streaming=True)

In [4]:
# functions for data
def load_datasets_from_paths(paths):
    datasets_list = []

    # Load each dataset from the provided paths
    for path in paths:
        dataset = load_dataset('text', data_files={"train": path}, split='train', streaming=True)
        datasets_list.append(dataset)
    
    return datasets_list

def combine_and_batch_datasets(datasets, batch_size=1000):
    """Yield batches of items from a list of iterable datasets."""
    for dataset in datasets:
        batch = []
        for item in dataset:
            batch.append(item['text'])
            if len(batch) == batch_size:
                yield batch
                batch = []
        # Yield any remaining items in the current dataset that didn't complete a full batch
        if batch:
            yield batch

#paths = [str(x) for x in Path('/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR').glob("**/*.txt")]


#combined_generator = combine_and_batch_datasets(load_datasets_from_paths(paths))

In [37]:
paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_1_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_2_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_3_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_4_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_5_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_6_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_7_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_8_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_9_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_10_converted.txt']



p = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_1_converted.txt']


combined_generator1 = combine_and_batch_datasets(load_datasets_from_paths(p))
combined_generator = combine_and_batch_datasets(load_datasets_from_paths(paths))

## Train tokenizer

#### Option 1: train from old tokenizer

In [38]:
# # set vocabulary size
# vocab_size = 30000

# # select dataset to train with
# train_set = combined_generator
# # train_set = create_text_generator(oscar_short_it)

# # load an existing BPE tokenizer
# old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # retrain the tokenizer
# tokenizer = old_tokenizer.train_new_from_iterator(train_set, vocab_size)



In [39]:
tokenizer.save_pretrained('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10')

('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10/tokenizer_config.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10/special_tokens_map.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10/vocab.txt',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10/added_tokens.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10/tokenizer.json')

In [40]:
paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_1_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_2_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_3_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_4_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_5_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_6_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_7_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_8_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_9_converted.txt',
         '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_10_converted.txt']



p = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/help_tokenizer/nl_part_1_converted.txt']


combined_generator1 = combine_and_batch_datasets(load_datasets_from_paths(p))
combined_generator = combine_and_batch_datasets(load_datasets_from_paths(paths))

# set vocabulary size
vocab_size = 30000

# select dataset to train with
train_set = combined_generator
# train_set = create_text_generator(oscar_short_it)

# load an existing BPE tokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# retrain the tokenizer
tokenizer = old_tokenizer.train_new_from_iterator(train_set, vocab_size)


tokenizer.save_pretrained('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10')

('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/tokenizer_config.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/special_tokens_map.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/vocab.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/merges.txt',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/added_tokens.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10/tokenizer.json')

Save tokenizer

In [11]:
tokenizer.save_pretrained('/Users/jan/Documents/Master/Thesis/Code')

('/Users/jan/Documents/Master/Thesis/Code/tokenizer_config.json',
 '/Users/jan/Documents/Master/Thesis/Code/special_tokens_map.json',
 '/Users/jan/Documents/Master/Thesis/Code/vocab.json',
 '/Users/jan/Documents/Master/Thesis/Code/merges.txt',
 '/Users/jan/Documents/Master/Thesis/Code/added_tokens.json',
 '/Users/jan/Documents/Master/Thesis/Code/tokenizer.json')

#### Option 2: train from zero

In [5]:
# voor één file:
paths = path

# voor meerdere files:
#paths = [str(x) for x in Path("/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=20000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])



Save tokenizer

In [7]:
tokenizer.save_model('/Users/jan/Documents/Master/Thesis/Code/a')

['/Users/jan/Documents/Master/Thesis/Code/a/vocab.json',
 '/Users/jan/Documents/Master/Thesis/Code/a/merges.txt']

Load tokenizer

(Note: this is not necessary, as we will load it in transformers to actually tokenize data)

In [7]:

tokenizer = ByteLevelBPETokenizer(
    '/Users/jan/Documents/Master/Thesis/trained_tokenizer/vocab.json',
    '/Users/jan/Documents/Master/Thesis/trained_tokenizer/merges.txt',
)


## Tokenize data

### Load tokenizer

#### Option 1: BPE

In [5]:
class CustomBPE(RobertaTokenizerFast):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __call__(self, text, *args, **kwargs):

        # Call the parent class's __call__ method
        encoded = super().__call__(text, *args, **kwargs)
        
        
        # Add BOS and EOS tokens to the input_ids
        if 'input_ids' in encoded:
            encoded['input_ids'] = [self.bos_token_id] + encoded['input_ids'] + [self.eos_token_id]
        
        # If attention_mask is present, adjust it as well
        if 'attention_mask' in encoded:
            encoded['attention_mask'] = [1] + encoded['attention_mask'] + [1]
        
        return encoded



In [6]:
# # WITH special tokens in call output
tokenizer_bpe = CustomBPE.from_pretrained("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/tokenizeBPE/t50", max_length=512, truncation=True)

special_tokens = {
    'pad_token': '<pad>',
    'bos_token': '<s>',
    'eos_token': '</s>',
    'unk_token': '<unk>',
    'mask_token': '<mask>',
}

tokenizer_bpe.add_special_tokens(special_tokens)

# WITHOUT special tokens in call output
# tokenizer_bpe = RobertaTokenizerFast.from_pretrained("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/tokenizeBPE/t50", max_length=512, truncation=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'CustomBPE'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2

In [7]:
tokenizer_bpe.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [8]:

v = tokenizer_bpe.get_vocab()
g = {value: key for key, value in v.items()}

#### Option 2: WordPiece

In [9]:
from transformers import RobertaTokenizerFast


class CustomWP(RobertaTokenizerFast):
    def __init__(self, **kwargs):
        
        # Call the parent class's __init__ method with all keyword arguments
        super().__init__(**kwargs)


    def __call__(self, text, return_tensors=None, **kwargs):
        # Call the parent class's __call__ method to get the encoded result
        encoding = super().__call__(text, **kwargs)
        
        def trim_tokens(tokens):
            # Only trim if there are more than two tokens (to avoid empty sequences)
            return tokens[1:-1] if len(tokens) > 2 else tokens

        # Check and process input_ids
        if 'input_ids' in encoding:
            if isinstance(encoding['input_ids'][0], list):
                encoding['input_ids'] = [self.bos_token_id] + [trim_tokens(seq) for seq in encoding['input_ids']] + [self.eos_token_id]
            else:
                encoding['input_ids'] = [self.bos_token_id] + trim_tokens(encoding['input_ids']) + [self.eos_token_id]
        
        # # Check and process attention_mask
        # if 'attention_mask' in encoding:
        #     if isinstance(encoding['attention_mask'][0], list):
        #         encoding['attention_mask'] = [trim_tokens(mask) for mask in encoding['attention_mask']]
        #     else:
        #         encoding['attention_mask'] = trim_tokens(encoding['attention_mask'])

        if return_tensors == 'pt':
            encoding['input_ids'] = torch.tensor(encoding['input_ids'])
        return encoding










In [10]:

# WITHOUT special tokens in call output
tokenizer_wp = CustomWP.from_pretrained("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/tokenizeWP/t50", max_len=512)

# WITH special tokens in call output
#tokenizer_wp = RobertaTokenizerFast.from_pretrained("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/tokenizeWP/t50", max_len=512)

special_tokens = {
    'pad_token': '<pad>',
    'bos_token': '<s>',
    'eos_token': '</s>',
    'unk_token': '<unk>',
    'mask_token': '<mask>',
}

tokenizer_wp.add_special_tokens(special_tokens)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CustomWP'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


3

#### Option 3: Custom Tokenizer BPE

We must first define our own tokenizer class

In [143]:

import torch

class CustomTokenizerBPE:

    def __init__(self, segmentation_dictionary, bpe_tokenizer, max_length=512, pad_to_multiple_of=None):
        
        self.bpe_tokenizer = bpe_tokenizer
        self.bpe_vocab = self.bpe_tokenizer.get_vocab()

        self.segmentations = {word: seg for word, seg in segmentation_dictionary.items() if len(seg) > 0}
        self.seg_dict = {}
        for word, segs in self.segmentations.items():
            out = []
            for i, seg in enumerate(segs):
                if i == 0:
                    out.append('Ġ' + seg)
                else:
                    out.append(seg)
            self.seg_dict[word] = out
        
        self.segments = {seg for segs in self.seg_dict.values() for seg in segs}
        # self.seg_vocab = {seg: (i + len(self.bpe_vocab)) for i, seg in enumerate(self.segments) if not seg in self.bpe_vocab}
        
        # self.vocab = self.bpe_vocab | self.seg_vocab

        self.vocab = self.bpe_vocab.copy()
        
        for element in self.segments:
            if element not in self.vocab:
                self.vocab[element] = len(self.vocab) + 1

        self.mask_token = "<mask>"
        self.mask_token_id = self.vocab['<mask>']
        self.vocab[self.mask_token_id] = self.vocab['<mask>']
        
        self.cls_token = "<s>"
        self.cls_token_id = self.vocab['<s>']
        self.vocab[self.cls_token] = self.vocab['<s>']
        
        self.sep_token = "</s>"
        self.sep_token_id = self.vocab['</s>']
        self.vocab[self.sep_token] = self.vocab['</s>']
        
        self.pad_token = '<pad>'
        self.pad_token_id = self.vocab['<pad>']
        self.vocab[self.pad_token] = self.vocab['<pad>']
        


        # self.unk_token = '<|endoftext|>'
        # self.unk_token_id = self.vocab['<|endoftext|>']
        # self.vocab[self.unk_token] = self.vocab['<|endoftext|>']
        
        self.vocab['<unk>'] = len(self.vocab) + 1 
        self.unk_token = '<unk>'
        self.unk_token_id = self.vocab['<unk>']
        self.vocab[self.unk_token] = self.vocab['<unk>']
        
        # self.bos_token = '<|endoftext|>'
        # self.bos_token_id = self.vocab['<|endoftext|>']
        # self.vocab[self.bos_token] = self.vocab['<|endoftext|>']

        self.bos_token = '<s>'
        self.bos_token_id = self.vocab['<s>']
        self.vocab[self.bos_token] = self.vocab['<s>']
        
        self.eos_token = "</s>"
        self.eos_token_id = self.vocab["</s>"]
        self.vocab[self.eos_token] = self.vocab["</s>"]
        
        self.special_tokens = [self.vocab['<mask>'], self.vocab['<s>'], self.vocab['</s>'], self.vocab['<pad>'], self.vocab['<unk>']]
   
        
        self.vocab_size = len(self.vocab)

        self.inverted_vocab = {value: key for key, value in self.vocab.items()}

        self.max_length = max_length
        self.pad_to_multiple_of = pad_to_multiple_of

        


    def get_vocab(self):
        return self.vocab


    def tokenize(self, seq):

        # Implement your token to ID conversion logic here
        text = self.bpe_tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(seq)
        words_a = [word for word in [word for word, offset in text]]
        
        words_b = [word.replace('Ġ', '') for word in [word for word, offset in text]]
        tokens = []
        for i, word_b in enumerate(words_b):
            if word_b in self.seg_dict:
                tokens += self.seg_dict[word_b]
            else:
                if words_a[i][0] == 'Ġ':
                    tokens += self.bpe_tokenizer.tokenize(' ' + word_b)
                else: 
                    tokens += self.bpe_tokenizer.tokenize(word_b)    
        return tokens
        
        
    def encode(self, seq):


        return [self.bos_token_id] + self.convert_tokens_to_ids(self.tokenize(seq)) + [self.eos_token_id]

        


    def _convert_token_to_id(self, token):
        if token in self.vocab:
            return self.vocab[token]
        else:
            return self.unk_token_id

    
    def convert_tokens_to_ids(self, tokens):
        if isinstance(tokens, list):
            return [self._convert_token_to_id(token) for token in tokens]
        return self._convert_token_to_id(tokens)


    def _tokenize(self, seq):
        text = self.bpe_tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(seq)
        words_a = [word for word in [word for word, offset in text]]
        
        words_b = [word.replace('Ġ', '') for word in [word for word, offset in text]]
        tokens = []
        for i, word_b in enumerate(words_b):
            if word_b in self.seg_dict:
                tokens += self.seg_dict[word_b]
            else:
                if words_a[i][0] == 'Ġ':
                    tokens += self.bpe_tokenizer.tokenize(' ' + word_b)
                else: 
                    tokens += self.bpe_tokenizer.tokenize(word_b)    
        return tokens




    def __call__(self, text, return_tensors=None, padding=False, truncation=False, max_length=None, add_special_tokens=False):
        if isinstance(text, list):
            self.batch_encode_plus(text, truncation=truncation, return_tensors=return_tensors, padding=padding, max_length=max_length, add_special_tokens=add_special_tokens)
        else:

            token_ids = self.encode(text) 


            if truncation and max_length:

                if len(token_ids) > max_length:
                    token_ids = token_ids[:max_length - 1] + [self.eos_token_id]

            # if truncation and max_length:
            #     # token_ids = [self.bos_token_id] + token_ids[:max_length - 2] + [self.eos_token_id]
            #     # token_ids = token_ids[:max_length]
            #     token_ids = [self.bos_token_id] + token_ids[:max_length - 2] + [self.eos_token_id]

            if return_tensors == "pt":
                return {"input_ids": torch.tensor(token_ids, dtype=torch.long)}
            return {"input_ids": token_ids}




    def _convert_id_to_token(self, id):
        # Implement your ID to token conversion logic here
        return self.inverted_vocab[id]

    def convert_ids_to_tokens(self, ids):
        # Convert IDs back to tokens
        if isinstance(ids, list):
            return [self._convert_id_to_token(id) for id in ids]
        return self._convert_id_to_token(ids)
    
    def decode(self, ids):
        out = ''
        for id in ids:
            word = self._convert_id_to_token(id)
            if word[0] == 'Ġ':
                out += word.replace('Ġ', ' ')
            else:
                out += word
        return out
        

    def get_special_tokens_mask(self, token_ids, already_has_special_tokens=False):
        # Create a mask for special tokens
        return [1 if self._is_special_token(token_id) else 0 for token_id in token_ids]


    def _is_special_token(self, token_id):
        # Implement your logic to check if a token is a special token
        if token_id in self.special_tokens:
            return True
        else:
            return False


    def batch_encode_plus(self, texts, return_tensors=None, padding=False, truncation=False, max_length=None, add_special_tokens=False):
        batch_token_ids = [self.__call__(text, truncation=truncation, max_length=max_length, add_special_tokens=add_special_tokens)["input_ids"] for text in texts]
        
        if padding:
            if max_length is None:
                max_length = max(len(ids) for ids in batch_token_ids)
            batch_token_ids = [ids + [self.pad_token_id] * (max_length - len(ids)) for ids in batch_token_ids]
        
        if return_tensors == "pt":
            return {"input_ids": torch.tensor(batch_token_ids, dtype=torch.long)}
        return {"input_ids": batch_token_ids}


    def pad(self, batch, return_tensors="pt", pad_to_multiple_of=None):
        if pad_to_multiple_of is None:
            pad_to_multiple_of = self.pad_to_multiple_of


        input_ids_list = []
        for dictionary in batch:
            for key, value in dictionary.items():
                if isinstance(value, torch.Tensor):
                    input_ids_list.append(value.tolist())


        max_length = max(len(x) for x in input_ids_list)
        
        if pad_to_multiple_of is not None:
            max_length = (max_length + pad_to_multiple_of - 1) // pad_to_multiple_of * pad_to_multiple_of
        
        padded_batch = []
        for seq in input_ids_list:
            if len(seq) < max_length:
                seq.extend([self.pad_token_id] * (max_length - len(seq)))
            padded_batch.append(seq)
        
        attention_list = []
        for inner_list in padded_batch:
            p_list = [1 if value < self.vocab_size else 0 for value in inner_list]
            attention_list.append(p_list)
        
        if return_tensors == "pt":
            return {'input_ids': torch.tensor(padded_batch, dtype=torch.long), 'attention_mask': torch.tensor(attention_list, dtype=torch.long)}
        
        return {'input_ids': padded_batch, 'attention_mask': attention_list}

    
    def __len__(self):
        return self.vocab_size

    
        



Now we load the segmentation dictionary and help tokenizer

In [144]:
path_to_dict = '/Users/jan/Documents/Master/Thesis/Code/seg_dict.json'
path_to_tokenizer = "/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30-10"

import json

def load_json(path):
    with open(path, 'r') as f:
        my_dict = json.load(f)
    return my_dict


segmentation_dictionary= load_json(path_to_dict)
help_tokenizer = RobertaTokenizerFast.from_pretrained(path_to_tokenizer)


tokenizer_own_bpe = CustomTokenizerBPE(segmentation_dictionary, help_tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Option 4: Custom tokenizer WordPiece

In [111]:
class CustomTokenizerWP:

    def __init__(self, segmentation_dictionary, wp_tokenizer, max_length=512, pad_to_multiple_of=None):
        
        self.wp_tokenizer = wp_tokenizer
        self.wp_vocab = self.wp_tokenizer.get_vocab()

        self.segmentations = {word: seg for word, seg in segmentation_dictionary.items() if len(seg) > 0}
        self.seg_dict = {}
        for word, segs in self.segmentations.items():
            out = []
            for i, seg in enumerate(segs):
                if i == 0:
                    out.append(seg)
                else:
                    out.append('##' + seg)
            self.seg_dict[word] = out
        
        self.segments = {seg for segs in self.seg_dict.values() for seg in segs}
        # self.seg_vocab = {seg: (i + len(self.bpe_vocab)) for i, seg in enumerate(self.segments) if not seg in self.bpe_vocab}
        
        # self.vocab = self.bpe_vocab | self.seg_vocab

        self.vocab = self.wp_vocab.copy()
        
        for element in self.segments:
            if element not in self.vocab:
                self.vocab[element] = len(self.vocab) + 1
        

        self.l = len(self.vocab)

        self.vocab['<unk>'] = self.l + 1 
        self.unk_token = '<unk>'
        self.unk_token_id = self.vocab['<unk>']
        self.vocab[self.unk_token] = self.vocab['<unk>']

        self.vocab['<mask>'] = self.l + 2 
        self.mask_token = "<mask>"
        self.mask_token_id = self.vocab['<mask>']
        self.vocab[self.mask_token_id] = self.vocab['<mask>']
        
        self.vocab['<s>'] = self.l + 3
        self.cls_token = "<s>"
        self.cls_token_id = self.vocab['<s>']
        self.vocab[self.cls_token] = self.vocab['<s>']
        
        self.vocab['</s>'] = self.l + 4
        self.sep_token = "</s>"
        self.sep_token_id = self.vocab['</s>']
        self.vocab[self.sep_token] = self.vocab['</s>']
        
        self.vocab['<pad>'] = self.l + 5
        self.pad_token = '<pad>'
        self.pad_token_id = self.vocab['<pad>']
        self.vocab[self.pad_token] = self.vocab['<pad>']
        
        self.vocab['<s>'] = self.l + 6
        self.bos_token = '<s>'
        self.bos_token_id = self.vocab['<s>']
        self.vocab[self.bos_token] = self.vocab['<s>']
        
        self.vocab['</s>'] = self.l + 7 
        self.eos_token = "</s>"
        self.eos_token_id = self.vocab["</s>"]
        self.vocab[self.eos_token] = self.vocab["</s>"]
        
        self.special_tokens = [self.vocab['<mask>'], self.vocab['<s>'], self.vocab['</s>'], self.vocab['<pad>'], self.vocab['<unk>']]
   
        
        self.vocab_size = len(self.vocab)

        self.inverted_vocab = {value: key for key, value in self.vocab.items()}

        self.max_length = max_length
        self.pad_to_multiple_of = pad_to_multiple_of

        


    def get_vocab(self):
        return self.vocab


    def tokenize(self, seq):


        text = self.wp_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(seq)
        words_a = [word for word in [word for word, offset in text]]
        
        words_b = [word.replace('##', '') for word in [word for word, offset in text]]
        tokens = []
        for i, word_b in enumerate(words_b):
            if word_b in self.seg_dict:
                tokens += self.seg_dict[word_b]
            else:
                if words_a[i][0] == '##':
                    tokens += self.wp_tokenizer.tokenize(word_b)
                else: 
                    tokens += self.wp_tokenizer.tokenize(' ' + word_b)    
        return tokens
        
        
    def encode(self, seq):


        return [self.bos_token_id] + self.convert_tokens_to_ids(self.tokenize(seq)) + [self.eos_token_id]

        


    def _convert_token_to_id(self, token):
        if token in self.vocab:
            return self.vocab[token]
        else:
            return self.unk_token_id

    
    def convert_tokens_to_ids(self, tokens):
        if isinstance(tokens, list):
            return [self._convert_token_to_id(token) for token in tokens]
        return self._convert_token_to_id(tokens)


    def _tokenize(self, seq):

        text = self.wp_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(seq)
        words_a = [word for word in [word for word, offset in text]]
        
        words_b = [word.replace('##', '') for word in [word for word, offset in text]]
        tokens = []
        for i, word_b in enumerate(words_b):
            if word_b in self.seg_dict:
                tokens += self.seg_dict[word_b]
            else:
                if words_a[i][0] == '##':
                    tokens += self.wp_tokenizer.tokenize(word_b)
                else: 
                    tokens += self.wp_tokenizer.tokenize(' ' + word_b)    
        return tokens









    def __call__(self, text, return_tensors=None, padding=False, truncation=False, max_length=None, add_special_tokens=False):
        if isinstance(text, list):
            self.batch_encode_plus(text, truncation=truncation, return_tensors=return_tensors, padding=padding, max_length=max_length, add_special_tokens=add_special_tokens)
        else:

            token_ids = self.encode(text) 

            if truncation and max_length:

                if len(token_ids) > max_length:
                    token_ids = token_ids[:max_length - 1] + [self.eos_token_id]


            # if padding and max_length is not None:
            #     token_ids = token_ids + [self.pad_token_id] * (max_length - len(token_ids))





            if return_tensors == "pt":
                return {"input_ids": torch.tensor(token_ids, dtype=torch.long)}
            return {"input_ids": token_ids}









    def _convert_id_to_token(self, id):
        # Implement your ID to token conversion logic here
        return self.inverted_vocab[id]

    def convert_ids_to_tokens(self, ids):
        # Convert IDs back to tokens
        if isinstance(ids, list):
            return [self._convert_id_to_token(id) for id in ids]
        return self._convert_id_to_token(ids)
    
    def decode(self, ids):
        out = ''
        for id in ids:
            word = self._convert_id_to_token(id)
            if word[:2] == '##':
                out += word.replace('##', '')
            else:
                out += ' ' + word
        return out
        

    def get_special_tokens_mask(self, token_ids, already_has_special_tokens=False):
        # Create a mask for special tokens
        return [1 if self._is_special_token(token_id) else 0 for token_id in token_ids]


    def _is_special_token(self, token_id):
        # Implement your logic to check if a token is a special token
        if token_id in self.special_tokens:
            return True
        else:
            return False


    def batch_encode_plus(self, texts, return_tensors=None, padding=False, truncation=False, max_length=None, add_special_tokens=False):
        batch_token_ids = [self.__call__(text, truncation=truncation, max_length=max_length, add_special_tokens=add_special_tokens)["input_ids"] for text in texts]
        
        if padding:
            if max_length is None:
                max_length = max(len(ids) for ids in batch_token_ids)
            batch_token_ids = [ids + [self.pad_token_id] * (max_length - len(ids)) for ids in batch_token_ids]
        
        if return_tensors == "pt":
            return {"input_ids": torch.tensor(batch_token_ids, dtype=torch.long)}
        return {"input_ids": batch_token_ids}


    def pad(self, batch, return_tensors="pt", pad_to_multiple_of=None):
        if pad_to_multiple_of is None:
            pad_to_multiple_of = self.pad_to_multiple_of


        input_ids_list = []
        for dictionary in batch:
            for key, value in dictionary.items():
                if isinstance(value, torch.Tensor):
                    input_ids_list.append(value.tolist())


        #max_length = self.max_length or max(len(x) for x in input_ids_list)
        max_length = max(len(x) for x in input_ids_list)
        
        if pad_to_multiple_of is not None:
            max_length = (max_length + pad_to_multiple_of - 1) // pad_to_multiple_of * pad_to_multiple_of
        
        padded_batch = []
        for seq in input_ids_list:
            if len(seq) < max_length:
                seq.extend([self.pad_token_id] * (max_length - len(seq)))
            padded_batch.append(seq)
        
        attention_list = []
        for inner_list in padded_batch:
            p_list = [1 if value < self.vocab_size else 0 for value in inner_list]
            attention_list.append(p_list)
        
        if return_tensors == "pt":
            return {'input_ids': torch.tensor(padded_batch, dtype=torch.long), 'attention_mask': torch.tensor(attention_list, dtype=torch.long)}
        
        return {'input_ids': padded_batch, 'attention_mask': attention_list}

    
    def __len__(self):
        return self.vocab_size

In [112]:
path_to_dict = '/Users/jan/Documents/Master/Thesis/Code/seg_dict.json'
path_to_tokenizer = "/Users/jan/Documents/Master/Thesis/Code/Tokenizers/extra30wp-10"

import json

def load_json(path):
    with open(path, 'r') as f:
        my_dict = json.load(f)
    return my_dict


segmentation_dictionary = load_json(path_to_dict)
help_tokenizer = RobertaTokenizerFast.from_pretrained(path_to_tokenizer)

tokenizer_own_wp = CustomTokenizerWP(segmentation_dictionary, help_tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
len(tokenizer_own_wp)

50016

In [49]:
len(help_tokenizer)

30002

### Create dataset

#### Option 1: LineByLineTextDataset

In [248]:
%%time
from transformers import LineByLineTextDataset

dataset_a = LineByLineTextDataset(
    tokenizer=tokenizer_wp,
    file_path='/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_shorter.txt',
    block_size=128,
)

CPU times: user 397 ms, sys: 15.3 ms, total: 412 ms
Wall time: 89.6 ms


/Users/jan/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


#### Option 2: Dataset

In [145]:
%%time

import os
import torch
from datasets import Dataset


# pick tokenizer
tokenizer = tokenizer_own_bpe

# pick txt file to transform
text_file_paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_shorter2.txt']




# def tokenize function
def tokenize_line(line):
    # Specify the max_length parameter to ensure consistent padding
    return {'input_ids': tokenizer(line.strip(), add_special_tokens=True, truncation=True, max_length=512)['input_ids']}
    #return tokenizer(line, truncation=False, max_length=512, return_tensors='pt', add_special_tokens=False)

# , max_length=512, truncation=True, return_tensors='pt'

# Read the text file and tokenize each line

for text_file_path in text_file_paths:
    with open(text_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Create a list of dictionaries with tokenized lines
    # tokenized_lines = []
    # for line in lines:
    #     t = tokenize_line(line)
    #     if len

    
    
    
    tokenized_lines = [tokenize_line(line) for line in lines]
    
    print(tokenized_lines[0])
    print(tokenized_lines[1])
    print(tokenized_lines[2])

    dataset = Dataset.from_list(tokenized_lines).with_format("torch")

    base_name = text_file_path.strip('.txt')


    base_name = os.path.splitext(os.path.basename(text_file_path))[0]
    

    dataset_name = f'{base_name}_tokenized'
    p = f'/Users/jan/Documents/Master/Thesis/Code/Datasets/Tokenized/WP/{dataset_name}'
    print(dataset_name)

    #save
    #dataset.save_to_disk(p)

{'input_ids': [30002, 53, 40854, 698, 2528, 1325, 290, 5456, 84, 4886, 974, 532, 24079, 257, 14, 221, 52, 688, 642, 491, 314, 11741, 12, 626, 2105, 642, 4886, 3312, 47064, 749, 11445, 29745, 30000]}
{'input_ids': [30002, 24, 569, 356, 40415, 83, 6080, 257, 16672, 14, 221, 41, 78, 3648, 1938, 12, 642, 626, 2105, 4886, 3874, 569, 835, 7512, 12, 3312, 569, 8468, 626, 2105, 642, 293, 742, 84, 440, 40415, 492, 314, 1418, 6913, 532, 25616, 14, 30000]}
{'input_ids': [30002, 25, 221, 46, 533, 1301, 642, 14563, 374, 7421, 8832, 257, 26664, 356, 369, 495, 4993, 268, 12, 8184, 642, 9280, 257, 5701, 1418, 412, 674, 357, 824, 12, 769, 314, 4886, 500, 974, 11412, 257, 14, 30000]}
OSCAR_shorter2_tokenized
CPU times: user 1.75 s, sys: 16.2 ms, total: 1.76 s
Wall time: 1.77 s


In [146]:
def test_length(dataset):
    mx = 0
    for i in range(999):
        if dataset[i]['input_ids'].size()[0] > mx:
            mx = dataset[i]['input_ids'].size()[0]
            n = i
    print('max length:', mx)
    print('at nr', n)

test_length(dataset)


max length: 512
at nr 94


In [152]:
#dataset[255]

In [149]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [150]:
texts = [dataset[0], dataset[1]]

In [151]:
data_collator(texts)

{'input_ids': tensor([[30002,    53, 40854,   698,  2528,  1325,   290,  5456,    84,  4886,
            974,   532, 24079,   257,    14,   221,    52, 30003,   642,   491,
            314, 11741,    12,   626,  2105,   642,  4886,  3312,  3283,   749,
          32868, 30003, 30000, 30001, 30001, 30001, 30001, 30001, 30001, 30001,
          30001, 30001, 30001, 30001],
         [30002,    24,   569,   356, 40415,    83,  6080,   257, 16672,    14,
            221,    41, 30003,  3648,  1938,    12, 30003,   626,  2105,  4886,
           3874,   569,   835, 30003, 10981,  3312, 30003,  8468,   626,  2105,
          30003,   293,   742,    84,   440, 40415,   492,   314,  1418,  6913,
            532, 25616,    14, 30000]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1

In [93]:
{v: k for k, v in tokenizer.get_vocab().items()}[50001]

'<pad>'

In [ ]:
torch.set_printoptions(threshold=float('inf'))

In [1049]:
def find_largest_element(tensors):
    largest = float('-inf')  # Initialize to negative infinity
    for tensor in tensors:
        max_in_tensor = torch.max(tensor)
        if max_in_tensor > largest:
            largest = max_in_tensor.item()
    return largest

# List of tensors
tensors = [dd[i]['input_ids'] for i in range(1000)]

# Find and print the largest element
largest_element = find_largest_element(tensors)
print("The largest element is:", largest_element)

The largest element is: 50284


In [21]:
# def validate_collator_output(batch, vocab_size):

#     input_ids = batch['input_ids']
#     labels = batch['labels']
#     attention_mask = batch['attention_mask']
    
#     # check that all tensors have the same shape
#     if input_ids.shape != labels.shape or input_ids.shape != attention_mask.shape:
#         print(f"Shape mismatch: input_ids shape {input_ids.shape}, labels shape {labels.shape}, attention_mask shape {attention_mask.shape}")
#         return False
    
#     # check each tensor element
#     for i in range(input_ids.size(0)):  
#         for j in range(input_ids.size(1)):
#             input_id = input_ids[i, j].item()
#             label = labels[i, j].item()
#             attention = attention_mask[i, j].item()

#             # # check that labels match input_ids or are -100
#             # if label != -100 and input_id != tokenizer.mask_token_id:
#             #     print(f"Label mismatch at batch {i}, position {j}: label {label}, input_id {input_id}")
#             #     return False
            
#             # check that labels match input_ids or are -100
#             if label != -100 and input_id != tokenizer.mask_token_id:
#                 print(f"Label mismatch at batch {i}, position {j}: label {label}, input_id {input_id}")
#                 return False
            
#             # check that all input_ids and labels are within the vocabulary range
#             if input_id >= vocab_size or (label != -100 and label >= vocab_size):
#                 print(f"Out of range token at batch {i}, position {j}: input_id {input_id}, label {label}")
#                 return False
            
#             # check that attention_mask is 1 for non-padded tokens and 0 for padded tokens
#             if (input_id != tokenizer.pad_token_id and attention != 1) or (input_id == tokenizer.pad_token_id and attention != 0):
#                 print(f"Attention mask mismatch at batch {i}, position {j}: input_id {input_id}, attention {attention}")
#                 return False
    
#     print("Validation passed.")
#     return True



## Initialize model

Select tokenizer

In [157]:
tokenizer = tokenizer_own_wp

Select data collator

In [79]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

Select model configuration

In [159]:

config = RobertaConfig(
    vocab_size=len(tokenizer.get_vocab()),
    max_position_embeddings=512,
    num_attention_heads=8,
    num_hidden_layers=8,
    type_vocab_size=1, 
    hidden_size=512,
    intermediate_size=1024
)


model = RobertaForMaskedLM(config=config)

print(f'number of parameters model: {model.num_parameters()}')

number of parameters model: 38756096


## Set training arguments

In [1038]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/Users/jan/Documents/Master/Thesis/Code/Models/model3", # note: this is not where the model is saved, just info about training
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=1000,
    prediction_loss_only=True,
)


# training_args = {
#     "learning_rate": 5e-5,
#     "warmup_steps": 10000,  # 10% of 100,000 training steps
#     "total_training_steps": 100000,
#     "batch_size": 16,  # adjust based on GPU memory
#     "gradient_accumulation_steps": 2,  # effectively increases batch size to 32
#     "num_train_epochs": 3,  # adjust based on dataset and desired thoroughness
#     "optimizer": "AdamW",
#     "weight_decay": 0.01,
#     "gradient_clipping": 1.0,
#     "loss_function": "CrossEntropyLoss"
# }




# training_args = TrainingArguments(
#     output_dir="/Users/jan/Documents/Master/Thesis/Code/Models/model3",             # Directory to save the model and checkpoints
#     overwrite_output_dir=True,          # Overwrite the content of the output directory
#     num_train_epochs=1,                 # Number of training epochs
#     per_device_train_batch_size=32,     # Batch size per device during training
#     per_device_eval_batch_size=32,      # Batch size for evaluation
#     warmup_steps=10000,                 # Number of warmup steps for learning rate scheduler
#     weight_decay=0.01,                  # Strength of weight decay
#     logging_dir='./logs',               # Directory for storing logs
#     logging_steps=500,                  # Log every 500 steps
#     save_steps=10000,                   # Save checkpoint every 10,000 steps
#     save_total_limit=20,                # Limit the total amount of checkpoints and deletes the older checkpoints
#     learning_rate=5e-5,                 # Learning rate
#     gradient_accumulation_steps=2,      # Number of updates steps to accumulate before performing a backward/update pass
#     evaluation_strategy='steps',        # Evaluate every `eval_steps`
#     eval_steps=5000,                    # Evaluation and save checkpoints every 5,000 steps
#     gradient_checkpointing=False,       # Use gradient checkpointing to save memory at the cost of slower backward pass
#     fp16=True,                          # Use 16-bit (mixed) precision instead of 32-bit
#     report_to='tensorboard',            # Use TensorBoard to log training information
#     load_best_model_at_end=True,        # Load the best model when finished training
#     metric_for_best_model='accuracy',   # The metric to use to compare models
#     greater_is_better=True              # Whether the metric should be maximized
# )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [1039]:
# meer arguments


# training_args2 = TrainingArguments(
#     output_dir='./results',               # V
#     num_train_epochs=3,                   # V
#     per_device_train_batch_size=16,       # wat is hier goed? GPT raadt 16 aan om mee te beginnen
#     learning_rate=5e-5,                   # wat is hier goed? ga geen hyp param search doen
#     weight_decay=0.01,                    # Weight decay for regularization
#     logging_dir='./logs',                 # Directory for logs. Is dit nodig? 
#     logging_steps=500,                    # Frequency of logging. IS dit nodig?
#     save_steps=10_000,                    # Frequency of saving checkpoints
#     save_total_limit=2,                   # waarom zou je dit niet hoog zetten? denk dat geheugen geen probleem is
#     evaluation_strategy='steps',          # Evaluation strategy ('no', 'steps', 'epoch')
#     eval_steps=500,                       # Frequency of evaluation if strategy is 'steps'
#     load_best_model_at_end=True,          # Whether to load the best model at the end of training
# )


# # dit wordt aangeraden door GPT:
# training_args3 = TrainingArguments(
#     output_dir='./results',               # Directory to save model checkpoints and logs
#     num_train_epochs=3,                   # Number of training epochs
#     per_device_train_batch_size=16,       # Batch size for training on each device
#     per_device_eval_batch_size=16,        # Batch size for evaluation on each device
#     learning_rate=1e-4,                   # Initial learning rate
#     weight_decay=0.01,                    # Weight decay for regularization
#     logging_dir='./logs',                 # Directory for logs
#     logging_steps=500,                    # Frequency of logging
#     save_steps=5000,                      # Frequency of saving checkpoints
#     save_total_limit=2,                   # Maximum number of checkpoints to keep
#     evaluation_strategy='steps',          # Evaluation strategy ('no', 'steps', 'epoch')
#     eval_steps=5000,                      # Frequency of evaluation if strategy is 'steps'
#     load_best_model_at_end=True,          # Whether to load the best model at the end of training
#     fp16=True,                            # Enable mixed precision training
# )

## Train

In [42]:
# %%time
# trainer.train()

## Save

In [ ]:
trainer.save_model("/Users/jan/Documents/Master/Thesis/model-xx")

## Load & Test

In [22]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/Users/jan/Documents/Master/Thesis/model-xx",
    tokenizer=tokenizer
)

In [23]:
fill_mask("Ik ga morgen naar <mask>.")

[{'score': 0.03317979723215103,
  'token': 274,
  'token_str': ' de',
  'sequence': 'Ik ga morgen naar de.'},
 {'score': 0.027490602806210518,
  'token': 287,
  'token_str': ' van',
  'sequence': 'Ik ga morgen naar van.'},
 {'score': 0.022509176284074783,
  'token': 295,
  'token_str': ' een',
  'sequence': 'Ik ga morgen naar een.'},
 {'score': 0.01623355969786644,
  'token': 18,
  'token_str': '.',
  'sequence': 'Ik ga morgen naar..'},
 {'score': 0.015407562255859375,
  'token': 317,
  'token_str': ' voor',
  'sequence': 'Ik ga morgen naar voor.'}]

In [25]:
# if we do this for the untrained model:

from transformers import pipeline

fill_mask_untrained = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

fill_mask_untrained("Ik ga morgen naar <mask>.")


[{'score': 0.00024192493583541363,
  'token': 4352,
  'token_str': ' niemand',
  'sequence': 'Ik ga morgen naar niemand.'},
 {'score': 0.00020695451530627906,
  'token': 19344,
  'token_str': 'abine',
  'sequence': 'Ik ga morgen naarabine.'},
 {'score': 0.00020639803551603109,
  'token': 20510,
  'token_str': ' vetten',
  'sequence': 'Ik ga morgen naar vetten.'},
 {'score': 0.00020594659144990146,
  'token': 11583,
  'token_str': ' wapens',
  'sequence': 'Ik ga morgen naar wapens.'},
 {'score': 0.00020444301480893046,
  'token': 26612,
  'token_str': 'sules',
  'sequence': 'Ik ga morgen naarsules.'}]

We can see that the untrained model gives random tokens. Our very minimally trained model (only 100.000 lines of text) already predicts some more probable tokens.

## Train again

We want to train our model further. 

In [32]:
# reload model

from transformers import AutoConfig

model_dir = "/Users/jan/Documents/Master/Thesis/model-xx"

config = AutoConfig.from_pretrained(model_dir)

model_trained_reloaded = RobertaForMaskedLM.from_pretrained(
    model_dir,
    config=config,
    use_safetensors=True  # Ensure that the model is loaded from safetensors
)



In [35]:
# test whether the reloaded model returns the same predictions

fill_mask_reloaded = pipeline(
    "fill-mask",
    model=model_trained_reloaded,
    tokenizer=tokenizer
)

fill_mask_untrained2("Ik ga morgen naar <mask>.") == fill_mask("Ik ga morgen naar <mask>.")

True

We have succesfully loaded the trained model. We now want to train it further.

In [37]:


# reload the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("/home/scur2141/train_tokenizer1/tokenizer", max_len=512)


training_args = TrainingArguments(
    output_dir="/Users/jan/Documents/Master/Thesis/Code/Models/model2", # note, this is not where the model is saved, just info about training
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model_trained_reloaded, # BE SURE to take the reloaded model here
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)


AttributeError: Can't get attribute 'AcceleratorConfig' on <module 'transformers.trainer_pt_utils' from '/Users/jan/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/transformers/trainer_pt_utils.py'>

## Klad


In [117]:
def get_class_methods(cls):
    """
    This function returns a list of all methods in the given class.
    
    :param cls: The class to inspect.
    :return: A list of method names.
    """
    methods = [method_name for method_name in dir(cls) 
               if callable(getattr(cls, method_name))]
    return methods

In [626]:
with open('/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_short.txt') as input:
    with open('/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_shorter2.txt', 'w') as output:
        for i, line in enumerate(input):

            if i > 3000 and i < 4000:
                output.write(line)